# 12 - 双重稳健估计（Doubly Robust Estimation）

## 不要把所有的鸡蛋放在一个篮子里

我们已经学会了如何使用线性回归和倾向得分加权来估计 \\(E[Y|T=1] - E[Y|T=0] |X\\)。但是我们应该在什么时候使用哪一个呢？在不明确的情况下，请同时使用两者！双重稳健估计是一种将倾向得分和线性回归相结合的方法，您不必依赖它们中的任何一种。

为了了解这是如何工作的，让我们考虑一下心态实验。这是一项在美国公立高中进行的随机研究，旨在发现成长心态的影响。它的工作方式是学校邀请学生参加一个研讨会，向他们灌输一种成长的心态。然后，他们跟踪学生在大学期间的表现，并衡量他们在学业上的表现。这个衡量结果被编译为标准化的成就分数。为了保护学生的隐私，这项研究的真实数据没有公开。但是，我们有一个与 Athey 和 Wager 提供的统计属性相同的模拟数据集，因此我们将改为使用这个数据来进行分析。

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

style.use("fivethirtyeight")
pd.set_option("display.max_columns", 6)

In [2]:
data = pd.read_csv("./data/learning_mindset.csv")
data.sample(5, random_state=5)

,schoolid,intervention,achievement_score,...,school_ethnic_minority,school_poverty,school_size
259,73,1,1.480828,...,-0.515202,-0.169849,0.173954
3435,76,0,-0.987277,...,-1.310927,0.224077,-0.426757
9963,4,0,-0.152340,...,0.875012,-0.724801,0.761781
4488,67,0,0.358336,...,0.315755,0.054586,1.862187
2637,16,1,1.360920,...,-0.033161,-0.982274,1.591641


虽然这项研究做了随机化处理，但这些数据并不是没有出现混淆的情况。其中一个可能的原因是，干预变量是通过学生是否收到研讨会邀请来衡量的。因此，尽管被邀请参与的机会是随机的，但是否真的参与却不是。我们在这里处理一个不服从（non-compliance）的情况。这方面的一个证据是学生对成功的期望是如何与是否参加研讨会相关联的。自我报告中期望较高的学生更有可能参加成长心态研讨会。